In [4]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
import tensorflow as tf

import matplotlib.pyplot as plt

### Data load

In [2]:
url = 'https://raw.githubusercontent.com/JunetaeKim/DeepLearningClass/main/Dataset/sonar.csv'
df = pd.read_csv(url, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


### Data preprocessing

In [5]:
dataset = df.values
X = dataset[:, 0:60] # select input columns
Target = dataset[:, 60] # select the target column

e = LabelEncoder() # Instantiation
e.fit(Target) # Fitting
Y = e.transform(Target) # Converting string labels to numeric labels


### Model define

In [9]:
def ModelStruc():
    
    Inp = Input(X.shape[-1]) # input layer
    
    #----------------------  stack of hidden layers  -------------------
    Hidden = Dense(int(X.shape[-1] * 0.75), 'relu')(Inp)
    Hidden = Dense(int(X.shape[-1] * 0.5), 'relu')(Hidden)
    Hidden = Dense(int(X.shape[-1] * 0.25), 'relu')(Hidden)
   
    #----------------------  stack of hidden layers  -------------------
    
    Output = Dense(1, 'sigmoid')(Hidden)  # target layer
    
    sonar = Model(Inp, Output)
    sonar.compile(loss=tf.losses.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4), metrics='accuracy')
    
    return sonar

### k-fold cross-validation setting

In [25]:
n_fold = 5
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0) # Instantiation

accuracy = [] # null list to track performance

### Conduct k-fold cross-validation

In [26]:
for train, test in skf.split(X,Y):
    sonar = ModelStruc()
    sonar.fit(X[train].astype('float32'), Y[train], epochs=100, batch_size=10, shuffle=True, verbose=2)
    K_accuracy = np.round(sonar.evaluate(X[test].astype('float32'), Y[test])[1]* 100, 2)
    accuracy.append(K_accuracy)
    

Epoch 1/100
17/17 - 0s - loss: 0.7013 - accuracy: 0.4699
Epoch 2/100
17/17 - 0s - loss: 0.6892 - accuracy: 0.5602
Epoch 3/100
17/17 - 0s - loss: 0.6836 - accuracy: 0.6145
Epoch 4/100
17/17 - 0s - loss: 0.6801 - accuracy: 0.6687
Epoch 5/100
17/17 - 0s - loss: 0.6764 - accuracy: 0.7229
Epoch 6/100
17/17 - 0s - loss: 0.6726 - accuracy: 0.7410
Epoch 7/100
17/17 - 0s - loss: 0.6678 - accuracy: 0.7530
Epoch 8/100
17/17 - 0s - loss: 0.6631 - accuracy: 0.7711
Epoch 9/100
17/17 - 0s - loss: 0.6588 - accuracy: 0.7651
Epoch 10/100
17/17 - 0s - loss: 0.6540 - accuracy: 0.7349
Epoch 11/100
17/17 - 0s - loss: 0.6469 - accuracy: 0.7651
Epoch 12/100
17/17 - 0s - loss: 0.6386 - accuracy: 0.7771
Epoch 13/100
17/17 - 0s - loss: 0.6292 - accuracy: 0.7831
Epoch 14/100
17/17 - 0s - loss: 0.6255 - accuracy: 0.7771
Epoch 15/100
17/17 - 0s - loss: 0.6093 - accuracy: 0.7831
Epoch 16/100
17/17 - 0s - loss: 0.6009 - accuracy: 0.7711
Epoch 17/100
17/17 - 0s - loss: 0.5895 - accuracy: 0.7831
Epoch 18/100
17/17 - 0s

### Check the results

In [27]:
accuracy

[78.57, 85.71, 76.19, 85.37, 80.49]